In [3]:
#import the package we need
import pandas as pd
import numpy as np
import pylab as pl 
import statsmodels.api as sm
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import math

# 1 Introduction to the Logistic Rgression Model

## 1.2 Fitting the Logistic Regression Model

In [6]:
#read the data sets CHDAGE
data = pd.read_table('D:/GitHub/applied_logistic_regression/data/CHDAGE/CHDAGE/CHDAGE.txt',header=0,encoding='gb2312',index_col='ID')
data.head()

,AGE,CHD
ID,,
1,20,0
2,23,0
3,24,0
5,25,1
4,25,0


In [9]:
target=data['CHD']
var=data[['AGE']]
var = sm.add_constant(var)

In [10]:
logit = sm.Logit(target, var)
result = logit.fit()
print(result.summary())

Optimization terminated successfully.
         Current function value: 0.536765
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:                    CHD   No. Observations:                  100
Model:                          Logit   Df Residuals:                       98
Method:                           MLE   Df Model:                            1
Date:                Tue, 01 Aug 2017   Pseudo R-squ.:                  0.2145
Time:                        22:37:08   Log-Likelihood:                -53.677
converged:                       True   LL-Null:                       -68.331
                                        LLR p-value:                 6.168e-08
                 coef    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
const         -5.3095      1.134     -4.683      0.000        -7.531    -3.088
AGE            0.1109      0.

In [ ]:
#输出fitted values 和 logit

# 3 Interpretation of the Fitted Logistic Regression


## 3.6 Presentation and Interpretation of the Fitted Values


In [50]:
#read the data sets described in Section 1.6 the Burn Study
data = pd.read_table('D:/GitHub/applied_logistic_regression/data/BURN/BURN1000.txt',header=0,encoding='gb2312',index_col='ID')

In [27]:
data.head()

,FACILITY,DEATH,AGE,GENDER,RACEC,TBSA,INH_INJ,FLAME
ID,,,,,,,,
1,11,0,26.6,1,1,25.3,0,1
2,1,0,2.0,0,0,5.0,0,0
3,12,0,22.0,0,0,2.0,0,0
4,1,0,37.3,1,1,2.0,0,0
5,1,0,52.1,1,1,6.0,0,1


In [59]:
target=data['DEATH']
var=data[['TBSA','INH_INJ']]
var = sm.add_constant(var)
var.head()

,const,TBSA,INH_INJ
ID,,,
1,1,25.3,0
2,1,5.0,0
3,1,2.0,0
4,1,2.0,0
5,1,6.0,0


In [60]:
logit = sm.Logit(target, var)

In [1]:
result = logit.fit()
print(result.summary())

NameError: name 'logit' is not defined

# 4 Purpose Selection of Covariates

### 4.2.2 Examples of Purposeful Selection

** Example 2: The Burn Injury Study **

In [6]:
#read the BURN1000 data
BURN1000 = pd.read_table('D:/GitHub/applied_logistic_regression/data/BURN/BURN1000.txt',header=0,encoding='gb2312',index_col='ID')

#show the data
print(BURN1000.head())


    FACILITY  DEATH   AGE  GENDER  RACEC  TBSA  INH_INJ  FLAME
ID                                                            
1         11      0  26.6       1      1  25.3        0      1
2          1      0   2.0       0      0   5.0        0      0
3         12      0  22.0       0      0   2.0        0      0
4          1      0  37.3       1      1   2.0        0      0
5          1      0  52.1       1      1   6.0        0      1


In [23]:
#fitting themodel containing all covariates
death=BURN1000['DEATH']
varX=BURN1000.loc[:,['AGE', 'TBSA', 'GENDER', 'RACEC', 'INH_INJ', 'FLAME']]
#varX=BURN1000.drop(['DEATH'],1)

varX = sm.add_constant(varX)
varX.head()

,const,AGE,TBSA,GENDER,RACEC,INH_INJ,FLAME
ID,,,,,,,
1,1,26.6,25.3,1,1,0,1
2,1,2.0,5.0,0,0,0,0
3,1,22.0,2.0,0,0,0,0
4,1,37.3,2.0,1,1,0,0
5,1,52.1,6.0,1,1,0,1


In [24]:
logit = sm.Logit(death, varX)
result = logit.fit()
print(result.summary())

Optimization terminated successfully.
         Current function value: 0.168231
         Iterations 9
                           Logit Regression Results                           
Dep. Variable:                  DEATH   No. Observations:                 1000
Model:                          Logit   Df Residuals:                      993
Method:                           MLE   Df Model:                            6
Date:                Wed, 09 Aug 2017   Pseudo R-squ.:                  0.6020
Time:                        22:58:30   Log-Likelihood:                -168.23
converged:                       True   LL-Null:                       -422.71
                                        LLR p-value:                9.893e-107
                 coef    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
const         -7.6952      0.691    -11.133      0.000        -9.050    -6.340
AGE            0.0829      0.

We can see that the Wald test for GENDER and FLAME is not significant. We remove both vatiable.(See the book page 116 for details)

In [26]:
death=BURN1000['DEATH']
varX=BURN1000.loc[:,['AGE', 'TBSA', 'RACEC', 'INH_INJ', ]]
#varX=BURN1000.drop(['DEATH'],1)

varX = sm.add_constant(varX)

logit = sm.Logit(death, varX)
result = logit.fit()
print(result.summary())

Optimization terminated successfully.
         Current function value: 0.169892
         Iterations 9
                           Logit Regression Results                           
Dep. Variable:                  DEATH   No. Observations:                 1000
Model:                          Logit   Df Residuals:                      995
Method:                           MLE   Df Model:                            4
Date:                Wed, 09 Aug 2017   Pseudo R-squ.:                  0.5981
Time:                        23:04:28   Log-Likelihood:                -169.89
converged:                       True   LL-Null:                       -422.71
                                        LLR p-value:                4.052e-108
                 coef    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
const         -7.5947      0.609    -12.471      0.000        -8.788    -6.401
AGE            0.0844      0.